In [60]:
import pandas as pd

In [61]:
# Load the datasets
tourism_df = pd.read_csv('API_ST.INT.ARVL_DS2_en_csv_v2_22950.csv', skiprows=4)
air_transport_df = pd.read_csv('API_IS.AIR.PSGR_DS2_en_csv_v2_122328.csv', skiprows=4)

In [90]:
# Filter data for East Africa
air_transport_df_east_africa = air_transport_df[air_transport_df['Country Name'] == 'Africa Eastern and Southern']
tourism_df_east_africa = tourism_df[tourism_df['Country Name'] == 'Africa Eastern and Southern']

# Reshape the data using melt, dropping NA values
air_transport_df_east_africa = air_transport_df_east_africa.melt(id_vars='Country Code', var_name='Years', value_name='Air_transport').dropna().reset_index()
tourism_df_east_africa = tourism_df_east_africa.melt(id_vars='Country Code', var_name='Years', value_name='Tourism').dropna().reset_index()

# Remove unwanted rows and set index
air_transport_df_east_africa = air_transport_df_east_africa.drop(['index', 'Country Code'], axis=1)[3:]
tourism_df_east_africa = tourism_df_east_africa.drop(['index', 'Country Code'], axis=1)[3:]

air_transport_df_east_africa = air_transport_df_east_africa.set_index('Years')
tourism_df_east_africa = tourism_df_east_africa.set_index('Years')

In [64]:
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score

print("🔗 Merging tourism and air transport datasets on 'Years'...")
combined_df_east_africa = tourism_df_east_africa.merge(air_transport_df_east_africa, how="inner", on="Years")
combined_df_east_africa


🔗 Merging tourism and air transport datasets on 'Years'...


,Tourism,Air_transport
Years,,
1995,11583544.70583,12153400.0
1996,13088654.204098,13178400.0
1997,13456245.973344,13456500.0
1998,14403851.832546,13234700.0
1999,15309377.842034,14038500.0
2000,15353177.301305,14963609.0
2001,15854696.286003,14546112.0
2002,17383375.415747,14462573.0
2003,17844385.282024,15802323.0


In [69]:
print("🔄 Converting 'Tourism' and 'Air_transport' columns to float...")
combined_df_east_africa["Tourism"] = combined_df_east_africa["Tourism"].astype(float)
combined_df_east_africa["Air_transport"] = combined_df_east_africa["Air_transport"].astype(float)

🔄 Converting 'Tourism' and 'Air_transport' columns to float...


In [72]:
print("📊 Performing Pearson correlation test...")

correlation, p_value = stats.pearsonr(
    combined_df_east_africa["Tourism"],
    combined_df_east_africa["Air_transport"]
)

📊 Performing Pearson correlation test...


In [73]:
X = combined_df_east_africa["Air_transport"]
y = combined_df_east_africa["Tourism"]


In [74]:
print("📈 Performing linear regression...")
slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)
print(f"📐 Linear Regression Equation: y = {intercept:.4f} + {slope:.4f} * x\n")

📈 Performing linear regression...
📐 Linear Regression Equation: y = 6012334.9555 + 0.7216 * x



In [75]:
def create_model(x):
    return slope * x + intercept

model_line = list(map(create_model, X))

In [76]:
mse = mean_squared_error(y, model_line)
rmse = r2_score(y, model_line)
print(f"📏 Mean Squared Error (MSE): {mse:.4f}")
print(f"📊 R-squared Score (R²): {rmse:.4f}\n")

📏 Mean Squared Error (MSE): 4673954372100.4277
📊 R-squared Score (R²): 0.9485



In [ ]:
print("🔮 Predicting tourism for the year 2021...")


row_2021 = air_transport_df_east_africa.loc['2021'] if '2021' in air_transport_df_east_africa.index else None

if row_2021 is not None:
    air_transport_2021 = float(row_2021['Air_transport'])
    tourism_2021_predicted = create_model(air_transport_2021)
    print(f"📅 Predicted Tourism for 2021: {tourism_2021_predicted:.2f}")
else:
    print("⚠️ Data for 2021 not found in air transport dataset.")


🔮 Predicting tourism for the year 2021...
📅 Predicted Tourism for 2021: 23371368.23


In [84]:
combined_df_east_africa.to_csv('combined_df_east_africa.csv')

In [85]:
combined_df_east_africa

,Tourism,Air_transport
Years,,
1995,1.158354e+07,1.215340e+07
1996,1.308865e+07,1.317840e+07
1997,1.345625e+07,1.345650e+07
1998,1.440385e+07,1.323470e+07
1999,1.530938e+07,1.403850e+07
2000,1.535318e+07,1.496361e+07
2001,1.585470e+07,1.454611e+07
2002,1.738338e+07,1.446257e+07
2003,1.784439e+07,1.580232e+07
